In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import pymongo
from dotenv import load_dotenv

In [6]:
mongo_uri = os.getenv("MONGO_URI")
mongo_client = pymongo.MongoClient(mongo_uri)

db_cow_monitoring = mongo_client["cow_monitoring"]
collection_cow_data = db_cow_monitoring["CowData"]

In [72]:
cow_data = pd.DataFrame(list(collection_cow_data.find()))

In [73]:
cow_data_dict = {cow_id: cow_data[cow_data['slave_id'] == cow_id] for cow_id in cow_data['slave_id'].unique()}

In [74]:
# Function to create records
def create_record_from_data(cow_data):
    grouped = cow_data.groupby('timestamp')  # Group by timestamp
    records = []

    # For each group of records with the same timestamp
    for timestamp, group in grouped:
        record = {'timestamp': timestamp}
        
        for _, row in group.iterrows():
            record[row['parameter']] = row['value']
        
        records.append(record)
    
    return pd.DataFrame(records).ffill()

In [75]:
columns = ['timestamp', 'slave_id', 'body_temperature', 'activity_level', 'milk_production', 'body_condition_score', 'estrous_cycle', 'feed_intake', 'rumen_ph']
df = pd.DataFrame(columns=columns)
for i in cow_data_dict:
    new_record = create_record_from_data(cow_data_dict[i])
    new_record['slave_id'] = i
    df = pd.concat([df, new_record], ignore_index=True)

/var/folders/hm/g79r9phd7ld_128zswnn0qhh0000gn/T/ipykernel_63344/2729918074.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_record], ignore_index=True)


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Step 1: Simulate or load data
# Assuming you have a dataset with the following columns:
# Body Temperature (T), Activity Levels (A), Rumen pH (pH), Milk Production (M), Feed Intake (F), Body Condition Score (B), Estrus Heat Date (EHD)

# Simulated dataset
np.random.seed(42)  # For reproducibility
data_size = 1000
data = {
    "Body_Temperature": np.random.normal(38.5, 0.5, data_size),
    "Activity_Levels": np.random.normal(50, 10, data_size),
    "Rumen_pH": np.random.normal(6.5, 0.2, data_size),
    "Milk_Production": np.random.normal(20, 2, data_size),
    "Feed_Intake": np.random.normal(25, 3, data_size),
    "Body_Condition_Score": np.random.normal(3, 0.5, data_size),
    "Estrus_Heat_Date": np.random.randint(18, 25, data_size)  # Target variable
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Step 2: Split data into features and target
X = df.drop(columns=["Estrus_Heat_Date"])
y = df["Estrus_Heat_Date"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 4: Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)  # Regression output layer
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Step 6: Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# Step 7: Make predictions
y_pred = model.predict(X_test)

# Display a few predictions
for i in range(5):
    print(f"Actual: {y_test.iloc[i]}, Predicted: {y_pred[i][0]}")


ModuleNotFoundError: No module named 'tensorflow'